In [1]:
# import csv 
# reader = csv.reader(open('dataset.csv','r'))
# data = {}
# data["train"] = []
# data["val"] = []
# skip=0
# val=0
# for row in reader:
#     if skip==0:
#         skip=1
#         continue
#     else:
#         temp = {}
#         temp["text"] = row[1]
#         temp["label"] = row[2]
#         if val<1000:
#             val+=1
#             data["val"].append(temp)
#         else:
#             data["train"].append(temp)
# print(data["train"][0])

from datasets import load_dataset

data = load_dataset('csv', data_files='dataset.csv')
data = data["train"]
data

Using custom data configuration default-4491ed487bf8c8cd


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 613.92it/s]


Dataset csv downloaded and prepared to /home/vaibhav/.cache/huggingface/datasets/csv/default-4491ed487bf8c8cd/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 834.02it/s]


Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'text', 'label'],
    num_rows: 26709
})

In [2]:
data = data.shuffle(seed=40)
data
data.remove_columns([data.column_names[0]])
train_testvalid = data.train_test_split(test_size=0.1)
data = train_testvalid

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [5]:
tokenized_data = data.map(preprocess_function, batched=True)

100%|██████████| 3/3 [00:00<00:00, 35.79ba/s]


In [6]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer, return_tensors='tf')

2022-04-03 01:40:34.462081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-03 01:40:34.462101: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
import tensorflow as tf
tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask","input_ids","label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)
tf_val_set = tokenized_data["test"].to_tf_dataset(
    columns=["attention_mask","input_ids","label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)


2022-04-03 01:40:38.116557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-03 01:40:38.116586: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-03 01:40:38.116608: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (predator): /proc/driver/nvidia/version does not exist
2022-04-03 01:40:38.116921: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
from transformers import create_optimizer

batch_size = 16
train_epochs = 5
batches_per_epochs = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epochs*train_epochs)
optimizer, schedule = create_optimizer(init_lr = 2e-5, num_warmup_steps=0,num_train_steps = total_train_steps)

In [9]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)



2022-04-03 01:40:44.901253: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint 

In [10]:
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [12]:
history = model.fit(
    tf_train_set,
    validation_data=tf_val_set,
    epochs=train_epochs
)


Epoch 1/5
  21/1502 [..............................] - ETA: 16:35 - loss: 0.6802

KeyboardInterrupt: 